# UKB RAP: From BGEN File to Hail MatrixTable

This notebook shows how to load the BGEN files from UKB RAP using the latest version of Hail. Note that the current JupyterLab instance is one version behind. We used this community-developed app from the Lindren Lab to run this: https://github.com/lindgrengroup/hail-on-dnanexus

First, we initialize Hail. Note that the BGEN files are lz4 compressed, so we have to specify that in our Spark Context.

In [1]:
from pyspark.sql import SparkSession
import hail as hl
import hail.expr.aggregators as agg
import os

builder = (
    SparkSession
    .builder
    .enableHiveSupport()
    .config("spark.shuffle.mapStatus.compression.codec", "lz4") 
)
spark = builder.getOrCreate()
hl.init(sc=spark.sparkContext)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/cluster/dnax/jars/dnanexus-api-0.1.0-SNAPSHOT-jar-with-dependencies.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/cluster/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


2023-04-28 07:29:33.953 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-04-28 07:29:34.121 WARN  MetricsReporter:84 - No metrics configured for reporting
2023-04-28 07:29:34.123 WARN  LineProtoUsageReporter:48 - Telegraf configurations: url [metrics.push.telegraf.hostport], user [metrics.push.telegraf.user] or password [metrics.push.telegraf.password] missing.
2023-04-28 07:29:34.123 WARN  MetricsReporter:117 - metrics.scraping.httpserver.port


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2023-04-28 07:29:35.006 WARN  Utils:69 - Service 'sparkDriver' could not bind on port 42000. Attempting port 42001.
2023-04-28 07:29:35.775 WARN  Utils:69 - Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 43000. Attempting port 43001.


pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/backend/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/backend/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.2.0
SparkUI available at http://ip-10-60-133-121.eu-west-2.compute.internal:8081
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.108-48fb3a9bae04
LOGGING: writing to /home/dnanexus/hail-20230428-0729-0.2.108-48fb3a9bae04.log


Next, we'll find the files associated with field id 23159. This field corresponds to the Whole Exome Sequencing files in the BGEN format.

We first find the file ids using `dxpy.find_data_objects()`. 

Then we can do a `dx describe` on each of the file ids to get the relevant file information for each file, namely the file name and the folder path.

In [2]:
import dxpy, os
#import hail as hl
import pandas as pd
testing = True
project_dir = "/user/tladeras/"

field_id = "23159"

obj_gen = dxpy.find_data_objects(properties={"field_id":field_id}, name="*.bgen", name_mode="glob")

id_list = [a["id"] for a in obj_gen]

# Grab describe objects for every file id
out_describe = [dxpy.bindings.dxdataobject_functions.describe(id) for id in id_list]

Then we'll build a list of file paths using dxFUSE paths. Then we'll process and build the hdfs file paths for indexing, the file name, and the sample file locations. We only will use one of the sample file locations, since all of them are identical.

In [3]:
#make a list of file paths with /mnt/project prepended and using only basename of file
dxfuse_list = [f"file:///mnt/project/" +  desc["folder"] + "/" + os.path.splitext(desc["name"])[0] for desc in out_describe]

#grab file name
index_list = [f"" + desc["name"] for desc in out_describe]
#make list of hdfs locations for index
hdfs_list = [f"file:///mnt2/project" + project_dir + "index/" + desc["name"] + ".idx2" for desc in out_describe]
#make list of bgen locations in project storage
bgen_list = [dx + ".bgen" for dx in dxfuse_list]
#make list of sample locations in project storage
sample_list = [f"" + dx + ".sample" for dx in dxfuse_list]

In order to transfer files, we'll have to reset some of the environment variables. We'll also mount the project as `/mnt2/project/` with dxFUSE's `-limitedWrite` option. We'll do this by using this script:

In [ ]:
%%bash
bash load-dxfuse.sh

Now we glue them together into a Pandas DataFrame, which we'll use to process all of the files. Note that if `testing == True`, then the notebook only processes the first 4 lines of the file manifest.

In [4]:
#glue everything into a Pandas DataFrame, which we'll use to process files
manifest = pd.DataFrame({"bgen": bgen_list, "sample": sample_list,"index":index_list, "hdfs": hdfs_list})  

if(testing):
    manifest = manifest.head(5)

manifest = manifest.tail(-1)    
    
file_out = "bgen_manifest.csv"
manifest.to_csv(file_out)

#save file manifest to project sstorage
dxpy.upload_local_file(filename=file_out, folder = "/user/tladeras/index/")

Here's what the file manifest looks like.

In [5]:
manifest

,bgen,sample,index,hdfs
1,file:///mnt/project//Bulk/Exome sequences/Popu...,file:///mnt/project//Bulk/Exome sequences/Popu...,ukb23159_c11_b0_v1.bgen,hdfs:///index/ukb23159_c11_b0_v1.bgen.idx2
2,file:///mnt/project//Bulk/Exome sequences/Popu...,file:///mnt/project//Bulk/Exome sequences/Popu...,ukb23159_c1_b0_v1.bgen,hdfs:///index/ukb23159_c1_b0_v1.bgen.idx2
3,file:///mnt/project//Bulk/Exome sequences/Popu...,file:///mnt/project//Bulk/Exome sequences/Popu...,ukb23159_c4_b0_v1.bgen,hdfs:///index/ukb23159_c4_b0_v1.bgen.idx2
4,file:///mnt/project//Bulk/Exome sequences/Popu...,file:///mnt/project//Bulk/Exome sequences/Popu...,ukb23159_c16_b0_v1.bgen,hdfs:///index/ukb23159_c16_b0_v1.bgen.idx2


Now we index each of the files by cycling over each BGEN file. This is currently saved in HDFS. We suggest storing the file manifest and the index files in a project so the BGEN files can be loaded in easily on subsequent runs.

Note the BGEN files are in genome build `GRCh37`. If you want to use them in build 38, the BGEN files will need to be lifted over to the new genome build.

In [6]:
for i, row in manifest.iterrows():
    hl.index_bgen(path=row["bgen"],
                  index_file_map={row["bgen"]:row["hdfs"]},
                  reference_genome="GRCh37",
                  contig_recoding=None,
                  skip_invalid_loci=True)

2023-04-28 07:32:24.282 Hail: INFO: Number of BGEN files indexed: 1             
2023-04-28 07:33:37.080 Hail: INFO: Number of BGEN files indexed: 1             
2023-04-28 07:34:04.005 Hail: INFO: Number of BGEN files indexed: 1             
2023-04-28 07:34:32.800 Hail: INFO: Number of BGEN files indexed: 1             


We can see the index files and their locations using `hdfs ds -ls`, which will list all of the files in `/index/`.

In [7]:
%%bash

hdfs dfs -ls /index/

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/cluster/hadoop/share/hadoop/common/lib/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/cluster/hadoop/share/hadoop/hdfs/lib/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


Found 4 items
drwxr-xr-x   - root supergroup          0 2023-04-28 07:32 /index/ukb23159_c11_b0_v1.bgen.idx2
drwxr-xr-x   - root supergroup          0 2023-04-28 07:34 /index/ukb23159_c16_b0_v1.bgen.idx2
drwxr-xr-x   - root supergroup          0 2023-04-28 07:33 /index/ukb23159_c1_b0_v1.bgen.idx2
drwxr-xr-x   - root supergroup          0 2023-04-28 07:34 /index/ukb23159_c4_b0_v1.bgen.idx2


We should save these indexes into project storage.

%%bash

# download indexes to driver node
hdfs dfs get /index/ .
# upload the indexes to project storage
dx upload -r index/*.idx2 --destination /user/tladeras/index/

In [8]:
file_list = manifest["bgen"].tolist()
file_list

sample_list = manifest["sample"].tolist()[0]
sample_list

'file:///mnt/project//Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c11_b0_v1.sample'

In [9]:
map_dict = {f"" + row["bgen"]:f"" + row["hdfs"] for i, row in manifest.iterrows()}
map_dict

{'file:///mnt/project//Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c11_b0_v1.bgen': 'hdfs:///index/ukb23159_c11_b0_v1.bgen.idx2',
 'file:///mnt/project//Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c1_b0_v1.bgen': 'hdfs:///index/ukb23159_c1_b0_v1.bgen.idx2',
 'file:///mnt/project//Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c4_b0_v1.bgen': 'hdfs:///index/ukb23159_c4_b0_v1.bgen.idx2',
 'file:///mnt/project//Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c16_b0_v1.bgen': 'hdfs:///index/ukb23159_c16_b0_v1.bgen.idx2'}

In [10]:
#build index file dictionary    

#finally, import all bgen files
mt = hl.import_bgen(file_list,
                    entry_fields=['GT', 'GP'],
                    sample_file = sample_list,
                    n_partitions=None,
                    block_size=None,
                    variants=None,
                    index_file_map = map_dict)

2023-04-28 07:35:25.048 Hail: INFO: Number of BGEN files parsed: 4
2023-04-28 07:35:25.048 Hail: INFO: Number of samples in BGEN files: 469835
2023-04-28 07:35:25.048 Hail: INFO: Number of variants across all BGEN files: 6666112


In [11]:
mt.count()

(6666112, 469835)